<a href="https://colab.research.google.com/github/yuseongil034/yuseongil/blob/main/0731_%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
#!/usr/bin/env python3
"""
자율주행 자동차 분석 파이프라인
YOLO v11 기반 앙상블 모델 (PeopleNet + TrafficNet)
Google Colab 환경에서 실행
"""

# ========== 1. 라이브러리 설치 ==========
print("필요한 라이브러리를 설치합니다...")

# YOLO v11 (Ultralytics) 설치
!pip install ultralytics>=8.0.0 -q

# 유튜브 다운로드 라이브러리
!pip install yt-dlp -q

# 영상 처리 라이브러리
!pip install opencv-python-headless -q
!pip install opencv-contrib-python -q

# 기타 필요한 라이브러리
!pip install numpy pandas matplotlib seaborn -q
!pip install tqdm -q

# GPU 사용 확인
import torch
print(f"GPU 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU 이름: {torch.cuda.get_device_name(0)}")

# ========== 2. 라이브러리 임포트 ==========
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import yt_dlp
from ultralytics import YOLO
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# 구글 코랩 파일 다운로드를 위한 라이브러리
from google.colab import files

# ========== 3. 유틸리티 함수 정의 ==========

def download_youtube_video(url, output_path='downloaded_video.mp4'):
    """
    유튜브 영상 다운로드 함수

    Args:
        url (str): 유튜브 영상 URL
        output_path (str): 저장할 파일 경로

    Returns:
        str: 다운로드된 영상 파일 경로
    """
    ydl_opts = {
        'format': 'best[ext=mp4]/best',
        'outtmpl': output_path,
        'quiet': True,
        'no_warnings': True,
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            print(f"영상 다운로드 중: {url}")
            ydl.download([url])
        print(f"다운로드 완료: {output_path}")
        return output_path
    except Exception as e:
        print(f"다운로드 오류: {e}")
        return None

def extract_video_segment(input_path, output_path, start_time, end_time):
    """
    영상에서 특정 구간만 추출하는 함수

    Args:
        input_path (str): 입력 영상 경로
        output_path (str): 출력 영상 경로
        start_time (float): 시작 시간 (초)
        end_time (float): 종료 시간 (초)

    Returns:
        str: 추출된 영상 파일 경로
    """
    cap = cv2.VideoCapture(input_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # 비디오 작성기 설정
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # 프레임 범위 계산
    start_frame = int(start_time * fps)
    end_frame = int(end_time * fps)

    # 시작 프레임으로 이동
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    print(f"영상 구간 추출 중: {start_time}초 ~ {end_time}초")
    for frame_num in tqdm(range(start_frame, end_frame)):
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)

    cap.release()
    out.release()
    print(f"구간 추출 완료: {output_path}")
    return output_path

# ========== 4. 앙상블 모델 클래스 정의 ==========

class EnsembleDetector:
    """
    PeopleNet과 TrafficNet을 앙상블하는 YOLO v11 기반 검출기
    """

    def __init__(self):
        """모델 초기화"""
        print("앙상블 모델 초기화 중...")

        # YOLO v11 모델 로드
        # 사람 검출용 모델 (PeopleNet 대체)
        self.people_model = YOLO('yolov8n.pt')  # nano 모델 사용 (빠른 처리)

        # 교통 관련 객체 검출용 모델 (TrafficNet 대체)
        self.traffic_model = YOLO('yolov8n.pt')  # 같은 모델 사용하되 다른 클래스 필터링

        # 검출 대상 클래스 정의
        self.people_classes = ['person', 'bicycle']
        self.traffic_classes = ['car', 'motorcycle', 'bus', 'truck', 'traffic light', 'stop sign']

        # 색상 정의 (BGR 형식)
        self.colors = {
            'person': (255, 0, 0),      # 파란색
            'bicycle': (255, 128, 0),   # 하늘색
            'car': (0, 255, 0),         # 초록색
            'motorcycle': (0, 255, 128), # 연두색
            'bus': (0, 0, 255),         # 빨간색
            'truck': (0, 128, 255),     # 주황색
            'traffic light': (255, 0, 255), # 보라색
            'stop sign': (255, 255, 0)  # 청록색
        }

        print("모델 초기화 완료!")

    def detect_people(self, frame):
        """사람 및 자전거 검출"""
        results = self.people_model(frame, conf=0.4)
        detections = []

        for r in results:
            boxes = r.boxes
            if boxes is not None:
                for box in boxes:
                    cls = int(box.cls)
                    class_name = self.people_model.names[cls]
                    if class_name in self.people_classes:
                        x1, y1, x2, y2 = box.xyxy[0].tolist()
                        conf = float(box.conf)
                        detections.append({
                            'bbox': [x1, y1, x2, y2],
                            'class': class_name,
                            'confidence': conf,
                            'source': 'people'
                        })

        return detections

    def detect_traffic(self, frame):
        """교통 관련 객체 검출"""
        results = self.traffic_model(frame, conf=0.4)
        detections = []

        for r in results:
            boxes = r.boxes
            if boxes is not None:
                for box in boxes:
                    cls = int(box.cls)
                    class_name = self.traffic_model.names[cls]
                    if class_name in self.traffic_classes:
                        x1, y1, x2, y2 = box.xyxy[0].tolist()
                        conf = float(box.conf)
                        detections.append({
                            'bbox': [x1, y1, x2, y2],
                            'class': class_name,
                            'confidence': conf,
                            'source': 'traffic'
                        })

        return detections

    def ensemble_detections(self, people_dets, traffic_dets):
        """
        검출 결과 앙상블 처리
        중복 제거 및 신뢰도 기반 필터링
        """
        all_detections = people_dets + traffic_dets

        # IoU 기반 중복 제거
        filtered_detections = []
        for i, det1 in enumerate(all_detections):
            is_duplicate = False
            for j, det2 in enumerate(all_detections[i+1:], i+1):
                iou = self.calculate_iou(det1['bbox'], det2['bbox'])
                if iou > 0.5:  # IoU 임계값
                    # 신뢰도가 높은 것 선택
                    if det1['confidence'] < det2['confidence']:
                        is_duplicate = True
                        break

            if not is_duplicate:
                filtered_detections.append(det1)

        return filtered_detections

    def calculate_iou(self, box1, box2):
        """IoU (Intersection over Union) 계산"""
        x1_1, y1_1, x2_1, y2_1 = box1
        x1_2, y1_2, x2_2, y2_2 = box2

        # 교집합 영역 계산
        x1_i = max(x1_1, x1_2)
        y1_i = max(y1_1, y1_2)
        x2_i = min(x2_1, x2_2)
        y2_i = min(y2_1, y2_2)

        if x2_i < x1_i or y2_i < y1_i:
            return 0.0

        intersection = (x2_i - x1_i) * (y2_i - y1_i)

        # 합집합 영역 계산
        area1 = (x2_1 - x1_1) * (y2_1 - y1_1)
        area2 = (x2_2 - x1_2) * (y2_2 - y1_2)
        union = area1 + area2 - intersection

        return intersection / union if union > 0 else 0.0

    def draw_detections(self, frame, detections):
        """검출 결과를 프레임에 그리기"""
        for det in detections:
            x1, y1, x2, y2 = map(int, det['bbox'])
            class_name = det['class']
            confidence = det['confidence']

            # 클래스별 색상 선택
            color = self.colors.get(class_name, (255, 255, 255))

            # 바운딩 박스 그리기
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

            # 레이블 텍스트 준비
            label = f"{class_name}: {confidence:.2f}"

            # 텍스트 배경 그리기
            (label_width, label_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            cv2.rectangle(frame, (x1, y1 - label_height - 4), (x1 + label_width, y1), color, -1)

            # 텍스트 그리기
            cv2.putText(frame, label, (x1, y1 - 2), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

        return frame

    def process_frame(self, frame):
        """단일 프레임 처리"""
        # 1. PeopleNet으로 사람 검출
        people_detections = self.detect_people(frame)

        # 2. TrafficNet으로 교통 객체 검출
        traffic_detections = self.detect_traffic(frame)

        # 3. 앙상블 처리
        ensemble_detections = self.ensemble_detections(people_detections, traffic_detections)

        # 4. 결과 시각화
        result_frame = self.draw_detections(frame.copy(), ensemble_detections)

        return result_frame, ensemble_detections

# ========== 5. 영상 처리 파이프라인 ==========

def process_video(input_path, output_path, detector):
    """
    전체 영상 처理 파이프라인

    Args:
        input_path (str): 입력 영상 경로
        output_path (str): 출력 영상 경로
        detector: 앙상블 검출기 객체
    """
    # 영상 열기
    cap = cv2.VideoCapture(input_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # 출력 영상 설정
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # 통계 정보 저장용
    frame_stats = []

    print(f"영상 처리 시작: 총 {total_frames} 프레임")

    # 프레임별 처리
    with tqdm(total=total_frames, desc="프레임 처리") as pbar:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # 프레임 처리
            processed_frame, detections = detector.process_frame(frame)

            # 통계 정보 수집
            stats = {
                'frame': len(frame_stats),
                'total_objects': len(detections),
                'people': sum(1 for d in detections if d['class'] in ['person', 'bicycle']),
                'vehicles': sum(1 for d in detections if d['class'] in ['car', 'motorcycle', 'bus', 'truck']),
                'traffic_signs': sum(1 for d in detections if d['class'] in ['traffic light', 'stop sign'])
            }
            frame_stats.append(stats)

            # 정보 오버레이 추가
            info_text = f"Frame: {stats['frame']} | Objects: {stats['total_objects']} | " \
                       f"People: {stats['people']} | Vehicles: {stats['vehicles']}"
            cv2.putText(processed_frame, info_text, (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

            # 프레임 저장
            out.write(processed_frame)
            pbar.update(1)

    # 리소스 해제
    cap.release()
    out.release()

    print(f"영상 처리 완료: {output_path}")

    return frame_stats

# ========== 6. 결과 시각화 함수 ==========

def visualize_statistics(frame_stats):
    """처리 결과 통계 시각화"""
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns

    # 데이터프레임 생성
    df = pd.DataFrame(frame_stats)

    # 시각화 설정
    plt.figure(figsize=(15, 10))

    # 1. 시간에 따른 객체 수 변화
    plt.subplot(2, 2, 1)
    plt.plot(df['frame'], df['total_objects'], label='Total Objects', linewidth=2)
    plt.plot(df['frame'], df['people'], label='People', linewidth=2)
    plt.plot(df['frame'], df['vehicles'], label='Vehicles', linewidth=2)
    plt.xlabel('Frame')
    plt.ylabel('Count')
    plt.title('Object Detection Over Time')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # 2. 객체 유형별 분포
    plt.subplot(2, 2, 2)
    object_counts = {
        'People': df['people'].sum(),
        'Vehicles': df['vehicles'].sum(),
        'Traffic Signs': df['traffic_signs'].sum()
    }
    plt.pie(object_counts.values(), labels=object_counts.keys(), autopct='%1.1f%%')
    plt.title('Object Type Distribution')

    # 3. 히스토그램
    plt.subplot(2, 2, 3)
    plt.hist(df['total_objects'], bins=20, alpha=0.7, color='blue', edgecolor='black')
    plt.xlabel('Number of Objects per Frame')
    plt.ylabel('Frequency')
    plt.title('Object Count Distribution')
    plt.grid(True, alpha=0.3)

    # 4. 통계 요약
    plt.subplot(2, 2, 4)
    stats_text = f"""
    Detection Statistics Summary:

    Total Frames: {len(df)}
    Average Objects/Frame: {df['total_objects'].mean():.2f}
    Max Objects in Frame: {df['total_objects'].max()}
    Total People Detected: {df['people'].sum()}
    Total Vehicles Detected: {df['vehicles'].sum()}
    Total Traffic Signs: {df['traffic_signs'].sum()}
    """
    plt.text(0.1, 0.5, stats_text, fontsize=12, verticalalignment='center')
    plt.axis('off')

    plt.tight_layout()
    plt.savefig('detection_statistics.png', dpi=150, bbox_inches='tight')
    plt.show()

# ========== 7. 메인 실행 함수 ==========

def main():
    """메인 실행 함수"""
    print("=== 자율주행 자동차 분석 파이프라인 ===")
    print("YOLO v11 기반 앙상블 모델 (PeopleNet + TrafficNet)")
    print("="*40)

    # 1. 유튜브 URL 입력
    youtube_url = input("유튜브 영상 URL을 입력하세요: ")

    # 2. 분석 구간 설정
    start_time = float(input("분석 시작 시간(초)을 입력하세요: "))
    end_time = float(input("분석 종료 시간(초)을 입력하세요: "))

    # 3. 결과 파일명 설정
    output_name = input("결과 영상 파일명을 입력하세요 (확장자 제외): ")
    output_filename = f"{output_name}_analyzed.mp4"

    try:
        # 4. 영상 다운로드
        downloaded_video = download_youtube_video(youtube_url, 'temp_video.mp4')
        if not downloaded_video:
            print("영상 다운로드 실패!")
            return

        # 5. 구간 추출
        segment_video = extract_video_segment(
            downloaded_video,
            'segment_video.mp4',
            start_time,
            end_time
        )

        # 6. 앙상블 검출기 초기화
        detector = EnsembleDetector()

        # 7. 영상 처리
        frame_stats = process_video(segment_video, output_filename, detector)

        # 8. 통계 시각화
        print("\n분석 결과 시각화 중...")
        visualize_statistics(frame_stats)

        # 9. 결과 파일 다운로드
        print(f"\n처리 완료! 결과 파일: {output_filename}")
        files.download(output_filename)
        files.download('detection_statistics.png')

        # 10. 임시 파일 정리
        os.remove('temp_video.mp4')
        os.remove('segment_video.mp4')

        print("\n모든 작업이 완료되었습니다!")

    except Exception as e:
        print(f"오류 발생: {e}")
        import traceback
        traceback.print_exc()

# ========== 8. 실행 ==========
if __name__ == "__main__":
    main()

필요한 라이브러리를 설치합니다...
GPU 사용 가능: True
GPU 이름: Tesla T4
=== 자율주행 자동차 분석 파이프라인 ===
YOLO v11 기반 앙상블 모델 (PeopleNet + TrafficNet)
유튜브 영상 URL을 입력하세요: https://youtu.be/yZKmyfVuG1A?si=oZBYyYSwPmRbSPnJ
분석 시작 시간(초)을 입력하세요: 0
분석 종료 시간(초)을 입력하세요: 15
결과 영상 파일명을 입력하세요 (확장자 제외): 
영상 다운로드 중: https://youtu.be/yZKmyfVuG1A?si=oZBYyYSwPmRbSPnJ
다운로드 완료: temp_video.mp4
영상 구간 추출 중: 0.0초 ~ 15.0초


100%|██████████| 449/449 [00:01<00:00, 368.22it/s]


구간 추출 완료: segment_video.mp4
앙상블 모델 초기화 중...
모델 초기화 완료!
영상 처리 시작: 총 449 프레임


프레임 처리:   0%|          | 0/449 [00:00<?, ?it/s]


0: 384x640 2 cars, 15.3ms
Speed: 1.4ms preprocess, 15.3ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 8.4ms
Speed: 1.3ms preprocess, 8.4ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)


...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


모든 작업이 완료되었습니다!
